In [16]:
import numpy as np
import mat73
from IPython.display import display
import data_utils
import importlib
importlib.reload(data_utils)
from data_utils import nan_sanity_check, calculate_correlation
from plot_functions import reorganize_roi_gradient, reorder_hemispheres
import pickle
import pandas as pd
import scipy
import enigmatoolbox
from enigmatoolbox.utils.parcellation import parcel_to_surface
from enigmatoolbox.plotting import plot_cortical
import mne 
from mne.stats import fdr_correction


In [2]:
data_epi = mat73.loadmat("data//EPI_data.mat")
data_epi = data_epi['EPI_data']

data_HC = mat73.loadmat("data//HC_data.mat")
data_HC = data_HC['HC_data']

epi_labels = [int(value[2]) for value in data_epi]
key_labels = [f"sub{i+1}" for i in range(len(epi_labels))]
epi_labels_dict = dict(zip(key_labels, epi_labels))

del(data_epi); del(data_HC)

In [3]:
### Loading of the already calculated FOOOF exponents dictionaries ###
with open('fooof_data_HC_1_35.pkl', 'rb') as file:
    fooof_dict_HC = pickle.load(file)
with open('fooof_data_EPI_1_35.pkl', 'rb') as file:
    fooof_dict_EPI = pickle.load(file)

### checking for NaN values deriving from unsuccesful FOOOF calculations ###
nan_sanity_check(fooof_dict_EPI, 'EPI 1-35')
nan_sanity_check(fooof_dict_HC, 'HC 1-35')

### Loading of the scores of the epileptic patients in different clinical/neuropsychologica scores in a dataframe ###
measures_df_1_35 = pd.read_csv('data\\neuropsy_scores.csv', sep=';', decimal=",",thousands='.', na_values=['-','--','---'])

No Nan values present in group EPI 1-35
No Nan values present in group HC 1-35


In [11]:
### removing subjects already excluded from the epileptic patients time series ###

for idx in [2,18,45,55,66]:
  measures_df_1_35.drop(measures_df_1_35[measures_df_1_35['Number_of_SJ'] == idx].index, inplace=True)

measures_df_1_35.shape

In [37]:
list_fooof = [list(value) for value in fooof_dict_EPI.values()]
average_fooof_1_35 = np.array(list_fooof).mean(axis=0)

measures_df_1_35['fooof'] = [fooof_dict_EPI[f'sub{sub}'] for sub in measures_df_1_35['Number_of_SJ']]

In [38]:
### Creating a nested dict for every epiulepsy sub-diagnosis 
alpha = 0.05

measures_list = ['ieds','onset','duration','drugs','corsi','ltm_ve_imm','ltm_ve_diff','rey_copy','rey_reproduction','qit','tmta','tmtb']
significance_check = []

corrected_total_correlation_df = pd.DataFrame()

#sub_groups_descriptions = ['All TLE', 'Left TLE', 'Right TLE', 'Bilateral TLE']
columns_list = measures_df_1_35.columns[1:]

#inner loop for the measures of interes
for measure, column in zip(measures_list, columns_list):
  ###  RISOLVERE IL PROBLEMA DEL DROPNA E SIAMO A CAVALLO ###
  dropna_df = measures_df_1_35[['fooof', column]].dropna(subset = [column], inplace = False, axis = 0)
  np.shape(dropna_df)

  corrected_corr, corrected_p_value, corrected_mask = calculate_correlation(dropna_df['fooof'].tolist(), dropna_df[column].tolist(), type='spearman', alpha=alpha, fdr=True) 
  check = 0
  for roi in corrected_mask:
    if roi == 0:
      continue
    elif roi != 0:
      check = 1
      break
  significance_check.append(check)
  corrected_total_correlation_df[measure] = (corrected_corr, corrected_p_value, corrected_mask)

In [39]:
significant_scores = [measures_list[idx] for idx, value in enumerate(significance_check) if value == 1]
print(f'The only clinical/neuropsycological measures that significantly correlates with EPI FOOOF exponent in at least one ROI are {significant_scores}')

The only clinical/neuropsycological measures that significantly correlates with EPI FOOOF exponent in at least one ROI are ['drugs', 'ltm_ve_imm']


In [35]:
interactive = True
### to correctly plot the gradients obtained from BrainStorm data some indeces transformations are necessary with reorder_emispheres() and reorganize_roi_gradients()

# Map parcellated data to the surface
mat = []
# column value determines wich column of the correlation dataframe (that is, which score) is chosen for the plot
for vec in [corrected_total_correlation_df['drugs'][2], corrected_total_correlation_df['ltm_ve_imm'][2]]:
    corr_mat_plot = reorganize_roi_gradient(reorder_hemispheres(vec))
    corr_mat_plot = (parcel_to_surface(corr_mat_plot, 'aparc_fsa5'))
    mat.append(corr_mat_plot)

# Project the results on the surface brain
if interactive==True:
    # Project the results on the surface brain
    plot_cortical(array_name=mat, surface_name="fsa5", size=(4000, 2800), label_text= {'left':['Drugs','RAVLT imm recall']}, background=(1,1,1), transparent_bg = False, interactive = True, cmap='RdBu_r', color_bar=True, scale=(1,1), zoom = 1.25, color_range=(-0.49, 0.41))
else:
    plot_cortical(array_name=mat, surface_name="fsa5", size=(4000, 2800), screenshot= True, filename=f'IMMAGINI PER PAPER\\Spearman Correlation All TLE fooof vs {column} 1-35 Hz alpha {alpha} FDR corrected.jpeg', background=(1,1,1), transparent_bg = False, interactive = False, cmap='RdBu_r', color_bar=True, scale=(1,1), embed_nb = True, zoom = 1.25, color_range='sym')

### for the plot refer to the images folder ###